# 1. Setting

In [ ]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Setting results path
data_path='/content/drive/MyDrive/Dacon/뉴스 토픽 분류 AI/data/'
model_path='/content/drive/MyDrive/Dacon/뉴스 토픽 분류 AI/model/'
sub_path='/content/drive/MyDrive/Dacon/뉴스 토픽 분류 AI/sub/'

In [ ]:
# Load data
train=pd.read_csv(os.path.join(data_path,'train_data.csv'))
test=pd.read_csv(os.path.join(data_path,'test_data.csv'))
sample_submission=pd.read_csv(os.path.join(data_path,'sample_submission.csv'))

# 2. Making Back Translation Data(KOR->ENG)

In [ ]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

driver=chrome_setting()

In [ ]:
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
# Crawling takes long time 
# So it is essential to split whole data 
trans_list=[]
kor_to_trans(train['title'], 'en',0,10000)
np.save(data_path+'kor_to_eng_train_0_10000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',10000,20000)
np.save(data_path+'kor_to_eng_train_10000_20000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',20000,30000)
np.save(data_path+'kor_to_eng_train_20000_30000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',30000,40000)
np.save(data_path+'kor_to_eng_train_30000_40000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',40000,len(train))
np.save(data_path+'kor_to_eng_train_40000_all.npy',trans_list)

In [ ]:
eng_data0=np.load(data_path+'kor_to_eng_train_0_10000.npy')
eng_data1=np.load(data_path+'kor_to_eng_train_10000_20000.npy')
eng_data2=np.load(data_path+'kor_to_eng_train_20000_30000.npy')
eng_data3=np.load(data_path+'kor_to_eng_train_30000_40000.npy')
eng_data4=np.load(data_path+'kor_to_eng_train_40000_all.npy')

eng_data=[*eng_data0,*eng_data1,*eng_data2,*eng_data3,*eng_data4]
eng_data=pd.DataFrame(eng_data,columns=['eng_title'])

back_train=pd.concat([train,eng_data],axis=1)

## 2.1 Translate Chinese characters into English again

+ We have often identified cases in which translators do not translate  properly when sentences contain Chinese characters.

+ Therefore, it attempts to re-translate sentences containing Chinese characters with high frequency of appearance.

In [ ]:
# Check Chinese characters with high frequency of appearance
k=[]
for i in range(0,len(train)):
  a=re.findall('[一-龥]',back_train['title'][i])
  if len(a)!=0:
    k=[*k,*a]

Counter(k).most_common()[:35]

[('美', 1498),
 ('北', 1329),
 ('中', 795),
 ('朴', 661),
 ('日', 467),
 ('靑', 381),
 ('與', 291),
 ('英', 285),
 ('文', 184),
 ('野', 181),
 ('獨', 138),
 ('伊', 127),
 ('韓', 109),
 ('前', 95),
 ('佛', 92),
 ('檢', 73),
 ('軍', 69),
 ('安', 63),
 ('反', 54),
 ('行', 44),
 ('南', 37),
 ('硏', 27),
 ('故', 25),
 ('外', 24),
 ('亞', 23),
 ('對', 21),
 ('銀', 19),
 ('展', 19),
 ('重', 18),
 ('株', 18),
 ('新', 17),
 ('黃', 16),
 ('企', 14),
 ('詩', 14),
 ('車', 14)]

In [ ]:
# Save index containing Chinese characters
hanja_ind=[]
for i in range(0,len(train)):
  a=re.findall('[一-龥]',back_train['title'][i])
  if len(a)!=0:
    hanja_ind.append(i)

# Replace to korean
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

d = { "中": "중국", "美": "미국","北":"북한",'日':"일본",'英':'영국','行':'행','靑':'청와대','朴':'박','銀':'은행','與':'여당',
     '文':'문','野':'야당','獨':'독일','伊':'이탈리아','韓':'한국','佛':'프랑스','前':'전','檢':'검찰','軍':'군','安':'안철수','南':'남한',
     '亞':'아시아','展':'전시회','重':'차','株':'주식','詩':'시'}

In [ ]:
kr_title=[]
for i in back_train.iloc[hanja_ind,1]:
  temp=replace_all(i,d)
  kr_title.append(temp)

# Retranslation case (Chinese -> Korean)
print(len(kr_title))

6303


In [ ]:
# Reset selenium chrome driver
driver=chrome_setting()

def kor_to_trans_again(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kr_title.npy',trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
# Retranslating 
trans_list=[]
kor_to_trans_again(kr_title, 'en',0,len(kr_title))
np.save(data_path+'kr_title.npy',trans_list)

In [ ]:
# Sucessfully retranslated
kr_title=np.load(data_path+'kr_title.npy')
back_train.at[hanja_ind,'eng_title']=kr_title

back_train.iloc[hanja_ind,:]

,index,title,topic_idx,eng_title
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,It's going to be beyond Silicon Valley.Google ...
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,Special relationship between South Korean comp...
7,7,美대선 TV토론 음담패설 만회실패 트럼프…사과 대신 빌클린턴 공격해 역효과,4,U.S. presidential election debate failed to ma...
10,10,日 오키나와서 열린 강제징용 노동자 추도식,4,Memorial Ceremony for Forced Workers in Okinaw...
13,13,美올랜도 병원 최악 총기 테러 부상자 치료비 안 받는다,4,Orlando Hospital in the U.S. does not pay for ...
...,...,...,...,...
45603,45603,北 고난의 행군 이어 군자리정신까지…대북제재 효과,6,"Following North Korea's painful march, militar..."
45611,45611,北 연이은 도발에 靑 신속·단호 대응…내부선 수위조절 고민도,6,Cheong Wa Dae's quick and firm response to Nor...
45618,45618,실사판 옥자 나오나…中 돈육대란에 초대형 돼지 사육 붐,4,Is the real-life version of Okja coming out?Bi...
45623,45623,선거 유세 나선 日 입헌민주당 에다노 대표,4,"Edano, chairman of Japan's Constitutional Demo..."


## 2.2 Retry Untranslated Sentences

+ There are still cases that have not been translated properly
+ Using pororo because it is not translated using Papago

In [ ]:
# Translated sentences containing Korean : 3 case
hangul_ind=[]
for i in range(0,len(back_train)):
  temp=re.findall('[가-힣]',back_train['eng_title'][i])
  if len(temp)!=0:
    hangul_ind.append(i)

back_train.iloc[hangul_ind,3]

2341                      정부 북한 단체 수십곳·개인 수십명 독자 금융제재 방침종합
9461     Hong Joon-pyo conservative 궤멸하다 be destroyed t...
39198    기겁하다 be startled tourists and residents of the...
Name: eng_title, dtype: object

In [ ]:
# Modify them manually
back_train.at[2341,'eng_title']='The governments comprehensive financial sanctions on dozens of North Korean organizations and individuals.'
back_train.at[9461,'eng_title']=back_train['eng_title'][9461].replace('궤멸하다','').replace('  ','')
back_train.at[39198,'eng_title']=back_train['eng_title'][39198].replace('기겁하다','').replace('  ','')

In [ ]:
# There are still cases that have not been translated properly
back_train[back_train['eng_title']=='']

,index,title,topic_idx,eng_title
14,14,日 대기업 올해 평균 2.46% 임금 인상,4,
29,29,미국 산업생산 한달만에 0.1%↑…제조업 회복 기대,4,
42,42,유럽인 59% 난민이 테러 가능성 키운다 인식,4,
59,59,시위대에 실탄 발사 정국 불안 속 홍콩증시 2%대 하락,4,
126,126,아시안게임 경기장 잔디 점검한 김학범 중동팀에 유리한...,5,
...,...,...,...,...
45554,45554,건보공단 고객센터 직원 50% 열 나도 휴식 못해,2,
45559,45559,툴젠 최대주주에 제넥신…790억원에 지분 17% 취득,1,
45583,45583,청년 디지털 일자리 등 3차 추경 사업 민간 채용률 겨우 11%,2,
45584,45584,HMM 해원노조 97% 파업 찬성…31일 2차 조정으로 판가름,1,


In [ ]:
# Convert Chinese characters into Korean for sentences that have not been translated.
not_trans_index=back_train[back_train['eng_title']=='']['title'].index

not_transed=[]
for i in back_train.iloc[not_trans_index,1]:
  temp=replace_all(i,d)
  not_transed.append(temp)

In [ ]:
# Retranslating using pororo
mt = Pororo(task="translation", lang="multi")

not_trans=[]

for i in tqdm(range(0,len(not_transed))):
  not_trans.append(mt(not_transed[i], src="ko", tgt="en"))
  if i%100==0:
    np.save(data_path+'not_trans.npy',not_trans)
    print('{}th : '.format(i),not_trans[i])

np.save(data_path+'not_trans.npy',not_trans)

In [ ]:
not_trans=np.load(data_path+'not_trans.npy')
back_train.at[not_trans_index,'eng_title']=not_trans

back_train.iloc[not_trans_index,:]

,index,title,topic_idx,eng_title
14,14,日 대기업 올해 평균 2.46% 임금 인상,4,an average wage hike of 2.46% for large Japane...
29,29,미국 산업생산 한달만에 0.1%↑…제조업 회복 기대,4,Expectations for a recovery in the manufacturi...
42,42,유럽인 59% 난민이 테러 가능성 키운다 인식,4,The perception that 59% of European refugees r...
59,59,시위대에 실탄 발사 정국 불안 속 홍콩증시 2%대 하락,4,a 2% drop in the Hong Kong stock market amid t...
126,126,아시안게임 경기장 잔디 점검한 김학범 중동팀에 유리한...,5,"It's good for Kim Hak-beom's Middle East team,..."
...,...,...,...,...
45554,45554,건보공단 고객센터 직원 50% 열 나도 휴식 못해,2,I can't rest 50% of the staff at the Korea Hea...
45559,45559,툴젠 최대주주에 제넥신…790억원에 지분 17% 취득,1,Toolgen's largest shareholder with KRW 79 bill...
45583,45583,청년 디지털 일자리 등 3차 추경 사업 민간 채용률 겨우 11%,2,only 11% of the private employment rate of the...
45584,45584,HMM 해원노조 97% 파업 찬성…31일 2차 조정으로 판가름,1,approval of the 97% strike by the HMM Maritime...


In [ ]:
back_train.to_csv(data_path+'back_train.csv',index=False)

# 3. Making Back Translation Data(ENG->KOR)

In [ ]:
driver=chrome_setting()

def trans_to_kor(transed_list, transed_lang,start_index,final_index): 
  
  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(1.5)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'eng_to_kor_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=en&tk='+transed_lang+'&st='+transed_list[i])
      time.sleep(2)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.2)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
# Crawling takes long time 
# So it is essential to split whole data 
trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',0,10000)
np.save(data_path+'eng_to_kor_train_0_10000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',10000,20000)
np.save(data_path+'eng_to_kor_train_10000_20000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',20000,30000)
np.save(data_path+'eng_to_kor_train_20000_30000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',30000,40000)
np.save(data_path+'eng_to_kor_train_30000_40000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',40000,len(back_train))
np.save(data_path+'eng_to_kor_train_40000_all.npy',trans_list)

## 3.1 Retry Untranslated Sentences

+ If the proportion of korean language in the total sentence length is 0.6 or higher, it is considered an abnormal translation and an attempt to re-translate it.

+ Attempt to re-translate a translated sentence if the translated sentence has a ratio of less than 0.5 to the length of an existing sentence

In [ ]:
back0=np.load(data_path+'eng_to_kor_train_0_10000.npy')
back1=np.load(data_path+'eng_to_kor_train_10000_20000.npy')
back2=np.load(data_path+'eng_to_kor_train_20000_30000.npy')
back3=np.load(data_path+'eng_to_kor_train_30000_40000.npy')
back4=np.load(data_path+'eng_to_kor_train_40000_all.npy')

back_train_fin=[*back0,*back1,*back2,*back3,*back4]
back_train_fin=pd.DataFrame(back_train_fin,columns=['back_title'])

back_train_fin=pd.concat([train,back_train_fin],axis=1)
back_train_fin

,index,title,topic_idx,back_title
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천→항공편 취소...휴가여행자의 노여움.
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘 밸리를 넘어설 거야구글은 15조원을 들여 미국의 허브가 될 것이다.
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란의 대외 긴장 해법은 미국이 자국의 경제 전쟁을 중단하는 것이다.
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,뉴욕주 클린턴과 가까운 한국 기업 간 특수관계.공공 및 민간 업무와 결합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,"시진핑은 중국, 미국과 무역 협상의 조속한 타결을 희망한다."
...,...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1,미국 IB Stiffell과의 KB금융파트너쉽니다.선진 시장 공략
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2,"서울시교육청, 신종 코로나 바이러스 확산에 따른 휴업 및 연기 검토"
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1,게시판 키움증권 2020 키움 히어로 전시회 실투자기업
45652,45652,답변하는 배기동 국립중앙박물관장,2,배기동 국립중앙박물관장이 대답한다.


In [ ]:
# .?! 뒤 space가 없는 경우 공백추가(숫자사이는 제외) 후 '...' 처리 / 중복공백 제거
for i in range(0,len(back_train_fin)):
  back_train_fin.at[i,'back_title']=back_train_fin['back_title'][i].replace('U.S.','미국')
  back_train_fin.at[i,'back_title']=re.sub(r'([.?!/\\]+)(?![0-9])', r'\1 ', back_train_fin['back_title'][i]).replace('... ','...').replace('  ',' ').replace('.. ','..').strip()


# If the proportion of Hangul in the total sentence length is 0.6 or higher, it is considered an abnormal translation and an attempt to re-translate it.
# -> 전체 문장길이에서 한글이ㅁ 차지하는 비중이 0.6이상이면 이상번역으로 간주하고 재번역 시도

# Attempt to re-translate a translated sentence if the translated sentence has a ratio of less than 0.5 to the length of an existing sentence
# -> 번역된 문장이 기존 문장의 길이에 대한 비율이 0.5이하이면 재번역 시도
retrans_ind=[]

for i in tqdm(range(0,len(back_train_fin))):
  kor_ratio=len(re.sub('[a-zA-Z]','',back_train_fin['back_title'][i]))/(len(back_train_fin['back_title'][i])+1)
  if kor_ratio<0.6:
    retrans_ind.append(i)
  if len(back_train_fin['back_title'][i])/len(back_train_fin['title'][i])<=0.5:
    retrans_ind.append(i)

retrans_ind=list(set(retrans_ind))

In [ ]:
# Case that retranslation is necessary
back_train_fin.iloc[retrans_ind,:]

,index,title,topic_idx,back_title
40961,40961,한경연 3월 전일제 취업자 7.6% 감소…통계청 0.7%보다 심각,1,
24583,24583,김성태 댓글 조작 최순실 국정농단 사건과 빼다 박아,6,최순실 댓글 조작 사건
14,14,日 대기업 올해 평균 2.46% 임금 인상,4,
32782,32782,중소기업 88.5% 기업경영 위해 공동·협업 사업 필요,1,
32784,32784,LGU 1분기 영업익 3.7%↑…5G 과열로 수익압박 전망종합2보,1,
...,...,...,...,...
16371,16371,트럼프 주장대로 불법이민자 전원 추방시 미국 경제 2% 축소,4,
24564,24564,토니모리 연구개발업체 에이투젠 인수,1,토니몰리
24567,24567,V3 모바일 시큐리티 넉달 만에 100만 다운로드,0,V3 Mobile Security 4개월 만에 100만 다운로드
16381,16381,특징주 코디에스 화장품업체 합병 소식에 오름세,1,CODYS Cosmetics 주식회사 합병 소식에 상승.


In [ ]:
# Converting Chinese characters to Korean from re-translation targets
re_transed=[]
for i in back_train.iloc[retrans_ind,1]:
  temp=replace_all(i,d)
  re_transed.append(temp)

In [ ]:
# Retranslating using pororo
mt = Pororo(task="translation", lang="multi")

retransed=[]

for k,i in tqdm(enumerate(re_transed)):
  eng=mt(i, src="ko", tgt="en")
  ko=mt(ko, src="en", tgt="ko")

  retransed.append(ko)
  if k%100==0:
    np.save(data_path+'re_trans.npy',retransed)
    print('{}th : '.format(k),retransed[k])

np.save(data_path+'re_trans.npy',retransed)

In [ ]:
# Check retranslated data
retransed=np.load(data_path+'re_trans.npy')
back_train_fin.at[retrans_ind,'back_title']=retransed

# Successfully translated
back_train_fin.iloc[retrans_ind,:]

,index,title,topic_idx,back_title
40961,40961,한경연 3월 전일제 취업자 7.6% 감소…통계청 0.7%보다 심각,1,한국경제연구원은 3월 정규직이 7.6% 감소한 것으로 나타났다. 통계청이 0.7% ...
24583,24583,김성태 댓글 조작 최순실 국정농단 사건과 빼다 박아,6,국정농단 사건에서는 김성태 의원의 댓글 조작자 최순실 씨가 제외됐다.
14,14,日 대기업 올해 평균 2.46% 임금 인상,4,올해 일본 대기업의 평균 임금 인상률 2.46%
32782,32782,중소기업 88.5% 기업경영 위해 공동·협업 사업 필요,1,중소기업의 88.5% 경영을 위한 공동 및 협업 사업의 필요성
32784,32784,LGU 1분기 영업익 3.7%↑…5G 과열로 수익압박 전망종합2보,1,LGU의 1분기 영업이익은 3.7%인 5G 과열로 총 2대가 수익을 압박할 것으로 ...
...,...,...,...,...
16371,16371,트럼프 주장대로 불법이민자 전원 추방시 미국 경제 2% 축소,4,트럼프 대통령의 주장대로 모든 불법 이민자가 추방되면 미국 경제가 2% 감소한다.
24564,24564,토니모리 연구개발업체 에이투젠 인수,1,토니 모리 연구개발 기업 A2Gen 인수
24567,24567,V3 모바일 시큐리티 넉달 만에 100만 다운로드,0,V3 이동통신 보안 4개월 만에 100만대의 다운로드
16381,16381,특징주 코디에스 화장품업체 합병 소식에 오름세,1,전문 코디스 화장품 업체의 합병 소식을 전해라


In [ ]:
back_train_fin.to_csv(data_path+'back_train_fin.csv',index=False)